<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Все-задания-нужно-выполнить-только-с-помощью-sql" data-toc-modified-id="Все-задания-нужно-выполнить-только-с-помощью-sql-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Все задания нужно выполнить только с помощью sql</a></span></li></ul></div>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    SELECT CRIM as col, COUNT(CRIM) as empty
    FROM boston
    WHERE CRIM = 0 OR CRIM IS NULL
    UNION ALL
    SELECT ZN, COUNT(ZN)
    FROM boston
    WHERE ZN = 0 OR ZN IS NULL
    UNION ALL
    SELECT INDUS, COUNT(INDUS)
    FROM boston
    WHERE INDUS = 0 OR INDUS IS NULL
    UNION ALL
    SELECT CHAS, COUNT(CHAS)
    FROM boston
    WHERE CHAS = 0 OR CHAS IS NULL
    UNION ALL
    SELECT NOX, COUNT(NOX)
    FROM boston
    WHERE NOX = 0 OR NOX IS NULL
    """,
    con,
)

,col,empty
0,NaN,0
1,0.0,372
2,NaN,0
3,0.0,471
4,NaN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    SELECT CRIM AS Col, COUNT(DISTINCT CRIM) AS Uniqe
    FROM boston
    UNION ALL
    SELECT ZN AS Column, COUNT(DISTINCT ZN) AS Uniqe
    FROM boston
    UNION ALL
    SELECT INDUS AS Column, COUNT(DISTINCT INDUS) AS Uniqe
    FROM boston
    UNION ALL
    SELECT CHAS AS Column, COUNT(DISTINCT CHAS) AS Uniqe
    FROM boston
    UNION ALL
    SELECT NOX AS Column, COUNT(DISTINCT NOX) AS Uniqe
    FROM boston
    """,
    con,
)

,Col,Uniqe
0,0.00632,504
1,18.00000,26
2,2.31000,76
3,0.00000,2
4,0.53800,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [17]:
pd.read_sql(
    """
    SELECT col, mediana, min FROM(
    SELECT * FROM(
    SELECT 'ZN' AS col, ZN as mediana, MIN(ZN) OVER(ORDER BY ZN) as min, row_number() OVER(ORDER BY ZN) as row
    FROM boston)
    WHERE ROW = 506 / 2
    UNION ALL
    SELECT * FROM(
    SELECT 'CRIM', CRIM, MIN(CRIM) OVER(ORDER BY CRIM), row_number() OVER(ORDER BY CRIM) as row
    FROM boston)
    WHERE ROW = 506 / 2
    UNION ALL
    SELECT * FROM(
    SELECT 'INDUS', INDUS, MIN(INDUS) OVER(ORDER BY INDUS), row_number() OVER(ORDER BY INDUS) AS row
    FROM boston)
    WHERE ROW = 506 / 2
    UNION ALL
    SELECT * FROM(
    SELECT 'CHAS', CHAS, MIN(CHAS) OVER(ORDER BY CHAS), row_number() OVER(ORDER BY CHAS) AS row
    FROM boston)
    WHERE ROW = 506 / 2
    UNION ALL
    SELECT * FROM(
    SELECT 'NOX', NOX, MIN(NOX) OVER(ORDER BY NOX), row_number() OVER(ORDER BY NOX) AS row
    FROM boston)
    WHERE ROW = 506 / 2
    )
    WHERE mediana = min
    """,
    con,
)

,col,mediana,min
0,ZN,0.0,0.0
1,CHAS,0.0,0.0


In [ ]:
# Для ZN и CHAS медиана равна минимальному значению
# Низкая доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
# Небольшое количество земель, граничазих с рекой

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [36]:
pd.read_sql(
    """
    SELECT 25 as "Min_price_buildings", AVG(Av_RM) as Average_number_of_rooms_min_price,    
    (SELECT AVG(RM) 
    OVER(PARTITION BY MEDV 
    ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as Average_number_of_rooms_max_price,
    
    AVG(
    Av_RM-(SELECT AVG(RM) OVER(PARTITION BY MEDV ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Difference
    FROM (SELECT RM as Av_RM FROM boston ORDER BY MEDV LIMIT 25)
    
     UNION ALL
     
    SELECT 50 as "Min_price_buildings", AVG(Av_RM) as Average_number_of_rooms_min_price,    
    (SELECT AVG(RM) 
    OVER(PARTITION BY MEDV 
    ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as Average_number_of_rooms_max_price,
    
    AVG(
    Av_RM-(SELECT AVG(RM) OVER(PARTITION BY MEDV ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Difference
    FROM (SELECT RM as Av_RM FROM boston ORDER BY MEDV LIMIT 50)
    
    UNION ALL
    
    SELECT 100 as "Min_price_buildings", AVG(Av_RM) as Average_number_of_rooms_min_price,    
    (SELECT AVG(RM) 
    OVER(PARTITION BY MEDV 
    ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as Average_number_of_rooms_max_price,
    
    AVG(
    Av_RM-(SELECT AVG(RM) OVER(PARTITION BY MEDV ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Difference
    FROM (SELECT RM as Av_RM FROM boston ORDER BY MEDV LIMIT 100)
    
    UNION ALL
    
    
    SELECT 200 as "Min_price_buildings", AVG(Av_RM) as Average_number_of_rooms_min_price,    
    (SELECT AVG(RM) 
    OVER(PARTITION BY MEDV 
    ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as Average_number_of_rooms_max_price,
    
    AVG(
    Av_RM-(SELECT AVG(RM) OVER(PARTITION BY MEDV ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Difference
    FROM (SELECT RM as Av_RM FROM boston ORDER BY MEDV LIMIT 200)
    
    UNION ALL
    
    
    SELECT 300 as "Min_price_buildings", AVG(Av_RM) as Average_number_of_rooms_min_price,    
    (SELECT AVG(RM) 
    OVER(PARTITION BY MEDV 
    ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as Average_number_of_rooms_max_price,
    
    AVG(
    Av_RM-(SELECT AVG(RM) OVER(PARTITION BY MEDV ORDER BY MEDV) as Max_price
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Difference
    FROM (SELECT RM as Av_RM FROM boston ORDER BY MEDV LIMIT 300)
    """,
    con,
)

,Min_price_buildings,Average_number_of_rooms_min_price,Average_number_of_rooms_max_price,Difference
0,25,5.747840,7.484,-1.736160
1,50,5.753240,7.484,-1.730760
2,100,5.887120,7.484,-1.596880
3,200,5.911705,7.484,-1.572295
4,300,5.972227,7.484,-1.511773


In [ ]:
#Стоимость дома зависит от количества комнат

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [41]:
pd.read_sql(
    """
     SELECT LSTAT, MEDV, rang
     FROM (SELECT *, DENSE_RANK()
     OVER(ORDER BY lstat) rang
    FROM boston)
    WHERE MEDV = (SELECT max(MEDV)
    FROM boston)
    """,
    con,
)

,LSTAT,MEDV,rang
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,14
6,3.26,50.0,15
7,3.32,50.0,16
8,3.70,50.0,23
9,3.73,50.0,24


In [ ]:
# В дорогих домах ниже % более низкого статуса населения

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [44]:
pd.read_sql(
    """
    SELECT CASE WHEN CHAS = 1 THEN 'river' else 'no_river' end CHAS, AVG(MEDV) 
    FROM boston
    GROUP BY CHAS
    """,
    con,
)

,CHAS,AVG(MEDV)
0,no_river,22.093843
1,river,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [20]:
pd.read_sql(
    """
    SELECT * FROM(
    SELECT 'CRIM' as "row", (SELECT AVG(CRIM) FROM boston WHERE CHAS = 0) as "river-", 
    (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) as "river+"
    UNION ALL
    SELECT 'ZN' as "row", (SELECT AVG(ZN) FROM boston WHERE CHAS = 0) as "river-", 
    (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) as "river+"
    UNION ALL
    SELECT 'INDUS' as "row", (SELECT AVG(INDUS) FROM boston WHERE CHAS = 0) as "river-", 
    (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) as "river+"
    UNION ALL
    SELECT 'NOX' as "row", (SELECT AVG(NOX) FROM boston WHERE CHAS = 0) as "river-", 
    (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) as "river+"
    )
    WHERE "river-" < "river+"
    """,
    con,
)

,row,river-,river+
0,INDUS,11.019193,12.719143
1,NOX,0.551817,0.593426


In [ ]:
# Рядом с рекой больше неторговых предприятий (вероятно, производственные), поэтому выше уровень загрязнения воздуха

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

,proc,indus,nox
0,10,2.97,0.4280
1,20,4.49,0.4429
2,30,6.06,0.4840
3,40,7.38,0.5070
4,50,9.69,0.5380
5,60,13.89,0.5800
6,70,18.10,0.6090
7,80,18.10,0.6710
8,90,19.58,0.7130
9,100,27.74,0.8710


In [ ]:
#Прослеживается прямая связь между количеством пром. объектов и концентрацией азота